In [1]:
import pandas as pd
import numpy as np
import sklearn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import catboost
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [2]:
#read in data
train = pd.read_csv('train.csv', encoding='latin-1', index_col=0)
test = pd.read_csv('test.csv', encoding='latin-1', index_col=0)

/Users/hannahamilton/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
train.iloc[:548288,:].to_csv("train1.csv")
train.iloc[548288:1096576,:].to_csv("train2.csv")
train.iloc[1096576:1644864,:].to_csv("train3.csv")
train.iloc[1644864:2193152,:].to_csv("train4.csv")
test.to_csv("test.csv")

In [7]:
test.shape

(503469, 5)

In [5]:
x_train = train[[i for i in list(train.columns) if i != 'CERTIFIED']]
y_train = train.CERTIFIED
x_test = test[[i for i in list(test.columns) if i != 'CERTIFIED']]
y_test = test.CERTIFIED

In [4]:
list(train['STATE'].unique())

['NEW YORK',
 'CALIFORNIA',
 'OHIO',
 'RHODE ISLAND',
 'TEXAS',
 'ALABAMA',
 'MARYLAND',
 'MISSOURI',
 'NEVADA',
 'ARKANSAS',
 'SOUTH CAROLINA',
 'WASHINGTON',
 'NORTH CAROLINA',
 'MINNESOTA',
 'PENNSYLVANIA',
 'VIRGINIA',
 'LOUISIANA',
 'ILLINOIS',
 'COLORADO',
 'GEORGIA',
 'NEW JERSEY',
 'FLORIDA',
 'OKLAHOMA',
 'KANSAS',
 'INDIANA',
 'MASSACHUSETTS',
 'TENNESSEE',
 'ARIZONA',
 'ALASKA',
 'WISCONSIN',
 'MICHIGAN',
 'CONNECTICUT',
 'NORTH DAKOTA',
 'DISTRICT OF COLUMBIA',
 'MISSISSIPPI',
 'KENTUCKY',
 'NEW MEXICO',
 'OREGON',
 'NEBRASKA',
 'UTAH',
 'HAWAII',
 'PUERTO RICO',
 'MONTANA',
 'IOWA',
 'NEW HAMPSHIRE',
 'WYOMING',
 'VERMONT',
 'WEST VIRGINIA',
 'DELAWARE',
 'IDAHO',
 'SOUTH DAKOTA',
 'MAINE']

In [5]:
train.shape

(2193152, 5)

In [6]:
test.shape

(503469, 5)

In [5]:
approval_rates_states = []
#calculate approval rates for states
for state in list(train['STATE'].unique()):
    #get subset df
    df = train[train['STATE']==state]
    y = df['CERTIFIED']
    #calculate approval rate
    approval_rate_state = y.sum()/len(y)
    approval_rates_states.append(approval_rate_state)

In [6]:
approval_rates_states

[0.9539990601503759,
 0.9669024531010766,
 0.9761758875473058,
 0.9789578054423268,
 0.969458038789197,
 0.9506921760771504,
 0.9606635700066357,
 0.9775594321162413,
 0.9107142857142857,
 0.9803045500136774,
 0.9568595616374812,
 0.9800673707671657,
 0.9776618097895353,
 0.9767408095708229,
 0.9754907938056334,
 0.9680223858263004,
 0.9451476793248945,
 0.9745624114129477,
 0.9642664509169363,
 0.9732844352796847,
 0.9767479759592194,
 0.9449971865568572,
 0.9574895878213414,
 0.9711044520547946,
 0.9703182228987299,
 0.9675326238744162,
 0.9674788300690945,
 0.973358645783895,
 0.8914634146341464,
 0.9793264421473825,
 0.9716614277844389,
 0.9742721545335961,
 0.9531486146095718,
 0.9437151486988847,
 0.945273631840796,
 0.9693073893907806,
 0.9458823529411765,
 0.973610014661103,
 0.9688965301656768,
 0.9587088660658143,
 0.8905540417801998,
 0.8428927680798005,
 0.9227642276422764,
 0.9761820120867402,
 0.9775363382763177,
 0.8650306748466258,
 0.9647177419354839,
 0.95584239130434

In [7]:
approval_rates_soc_names = []
#calculate approval rates for soc names
for soc_name in list(train['SOC_NAME'].unique()):
    #get subset df
    df = train[train['SOC_NAME']==soc_name]
    y = df['CERTIFIED']
    #calculate approval rate
    approval_rate_soc_name = y.sum()/len(y)
    approval_rates_soc_names.append(approval_rate_soc_name)

In [8]:
approval_rates_soc_names

[0.9276899761853215,
 0.8193387398627573,
 0.8881457564575646,
 0.9106481481481481,
 0.8879581151832461,
 0.9631437637295582,
 0.9615651844365841,
 0.9049512532767813,
 0.9531739209547174,
 0.8571428571428571,
 0.9147172046194848,
 0.9783978397839784,
 0.9821675614793616,
 0.9823752056417749,
 0.9853947201030865,
 0.9656206924511473,
 0.8931560807483998,
 0.9342911153119092,
 0.928486742832507,
 0.9393752798925212,
 0.9490131578947368,
 0.9570802919708029,
 0.832225063938619,
 0.973316187108182,
 0.9170483460559796,
 0.9326264634415727,
 0.9391338275458447,
 0.8840423622329923,
 0.8913716814159292,
 0.9286150505530298,
 0.9596051435251332,
 1.0,
 0.9759046306969944,
 0.9755729094036164,
 0.9719062784349408,
 0.9726561187812826,
 0.8967889908256881,
 0.9664299960111687,
 0.9410299003322259,
 0.949349501073639,
 0.9610713880376802,
 0.962247355744057,
 0.9652761317847831,
 0.928796755295178,
 0.947579566729072,
 0.9541206457094308,
 0.9049786628733997,
 0.8379906852960746,
 0.86900742741

In [9]:
rates_states = pd.DataFrame()
rates_states['CATEGORY'] = list(train['STATE'].unique())
rates_states['RATE'] = approval_rates_states
rates_states.to_csv("rates_states.csv")

In [10]:
rates_soc_names = pd.DataFrame()
rates_soc_names['CATEGORY'] = list(train['SOC_NAME'].unique())
rates_soc_names['RATE'] = approval_rates_soc_names
rates_soc_names.to_csv("rates_soc_names.csv")

In [11]:
#replace state names with categories
states_group_1 = ['CALIFORNIA','OHIO','RHODE ISLAND','TEXAS','MISSOURI','ARKANSAS','WASHINGTON','NORTH CAROLINA','MINNESOTA','PENNSYLVANIA','VIRGINIA','ILLINOIS','GEORGIA','NEW JERSEY','KANSAS','INDIANA','MASSACHUSETTS','TENNESSEE','ARIZONA','WISCONSIN','MICHIGAN','CONNECTICUT','KENTUCKY','OREGON','NEBRASKA','IOWA','NEW HAMPSHIRE','DELAWARE','MAINE']
states_group_2 = ['NEW YORK','ALABAMA','MARYLAND','NEVADA','SOUTH CAROLINA','LOUISIANA','COLORADO','FLORIDA','OKLAHOMA','ALASKA','NORTH DAKOTA','DISTRICT OF COLUMBIA','MISSISSIPPI','NEW MEXICO','UTAH','HAWAII','PUERTO RICO','MONTANA','WYOMING','VERMONT','WEST VIRGINIA','IDAHO','SOUTH DAKOTA']

In [12]:
#replace soc_names with categories
soc_names_group_1 = ['Commercial and Industrial Designers','Postsecondary Education Teachers','Computer Hardware Engineers','Computer Programmers','Other Computer Occupations','Computer Systems Analysts and Engineers','Computer and Information Systems Managers','Life, Physical, and Social Scientists and Technicians','Computer and Information Research Scientists','Industrial and Manufacturing Engineers and Industrial and Manufacturing Engineering Technicians','Accountants and Auditors','Data Scientists and Data Science Technicians','Network and Computer Systems Administrators','Statisticians, Statistical Assistants, and Other Mathematical Occupations','Operations Research Analysts','Electrical Engineers and Electrical Engineering Technicians','Materials Scientists and Materials Engineers','Mechanical Engineers and Mechanical Engineering Technicians','Medical Scientists, Technicians, and Assistants','Occupational Therapists and Assistants','Software Quality Assurance Engineers and Testers']
soc_names_group_2 = ['Civil Engineers and Civil Engineering Technicians','Psychologists','Business Analysts','Other Artistic Workers','Sales and Related Workers','Other Health Professionals and Technicians','Primary and Secondary Education Teachers','Chemical Engineers','Marketing Managers','Financial Managers','Industrial Production Managers','Business Operations Specialists','Environmental Engineers and Environmental Engineering Technicians','Other Engineers and Engineering Technicians','Food Scientists and Food Science Technicians','Environmental Scientists and Environmental Technicians','Geologists, Geoscientists, and Geological Technicians','Fashion Designers','Dentists, Dental Assistants, Hygienists, and Laboratory Technicians','Physicians, Surgeons, Physician Assistants, and Surgical Assistants','Internists','Models','Pharmacists and Pharmacy Technicians','Purchasing Managers','Physical Therapists and Physical Therapist Assistants','Sales Managers','Special Education Teachers']
soc_names_group_3 = ['Community and Social Service Specialists','Coaches and Scouts','Other Managers','Compliance Officers','Construction Managers','Other Teachers and Teacher Assistants','Medical and Health Services Managers','Architects and Architectural Drafters','Lawyers','Other Designers','Graphic Designers','Interior Designers','Media and Communication Workers','Architectural and Engineering Managers','Public Relations and Fundraising Managers','Veterinarians, Veterinary Assistants, Veterinary Technicians, Laboratory Animal Caretakers, and Animal Scientists']
soc_names_group_4 = ['Other','Chief Executives','Paralegals and Legal Assistants','Office Clerks and Secretaries','Preschool Teachers']

In [13]:
#transform train data
#states
train.replace(states_group_1, 'STATES_GROUP_1', inplace=True)
train.replace(states_group_2, 'STATES_GROUP_2', inplace=True)

#soc_names
train.replace(soc_names_group_1, 'SOC_NAMES_GROUP_1', inplace=True)
train.replace(soc_names_group_2, 'SOC_NAMES_GROUP_2', inplace=True)
train.replace(soc_names_group_3, 'SOC_NAMES_GROUP_3', inplace=True)
train.replace(soc_names_group_4, 'SOC_NAMES_GROUP_4', inplace=True)

In [14]:
train.head()

,SOC_NAME,PREVAILING_WAGE,STATE,CERTIFIED,FULL_TIME
1,SOC_NAMES_GROUP_2,61602.0,STATES_GROUP_2,1,1
2,SOC_NAMES_GROUP_2,64792.0,STATES_GROUP_1,1,1
3,SOC_NAMES_GROUP_2,55432.0,STATES_GROUP_1,1,0
4,SOC_NAMES_GROUP_2,62608.0,STATES_GROUP_1,1,0
5,SOC_NAMES_GROUP_2,61256.0,STATES_GROUP_1,1,1


In [15]:
#transform test data
#states
test.replace(states_group_1, 'STATES_GROUP_1', inplace=True)
test.replace(states_group_2, 'STATES_GROUP_2', inplace=True)

#soc_names
test.replace(soc_names_group_1, 'SOC_NAMES_GROUP_1', inplace=True)
test.replace(soc_names_group_2, 'SOC_NAMES_GROUP_2', inplace=True)
test.replace(soc_names_group_3, 'SOC_NAMES_GROUP_3', inplace=True)
test.replace(soc_names_group_4, 'SOC_NAMES_GROUP_4', inplace=True)

In [16]:
test.head()

,SOC_NAME,PREVAILING_WAGE,STATE,CERTIFIED,FULL_TIME
1,SOC_NAMES_GROUP_1,102149.0,STATES_GROUP_1,1,1
2,SOC_NAMES_GROUP_1,56784.0,STATES_GROUP_1,1,1
3,SOC_NAMES_GROUP_1,85197.0,STATES_GROUP_1,1,1
4,SOC_NAMES_GROUP_1,117770.0,STATES_GROUP_1,1,1
5,SOC_NAMES_GROUP_1,66414.0,STATES_GROUP_1,1,1


In [17]:
train.iloc[:548288,:].to_csv("train_categorized1.csv")
train.iloc[548288:1096576,:].to_csv("train_categorized2.csv")
train.iloc[1096576:1644864,:].to_csv("train_categorized3.csv")
train.iloc[1644864:2193152,:].to_csv("train_categorized4.csv")
test.to_csv("test_categorized.csv")

In [18]:
train.shape

(2193152, 5)